In [10]:
import torch
print("🧠 PyTorch - GPU disponible:", torch.cuda.is_available())

import tensorflow as tf
print("⚙️ TensorFlow - GPU detectada:", tf.config.list_physical_devices('GPU'))


🧠 PyTorch - GPU disponible: True
⚙️ TensorFlow - GPU detectada: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [11]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Solo errores, no warnings ni info

In [12]:
# === Módulos estándar ===
import numpy as np
import random
import time

# === Keras ===
import tensorflow as tf
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense, Input # type: ignore
from tensorflow.keras.initializers import GlorotUniform # type: ignore
from tensorflow.keras.optimizers import SGD as KSGD, Adam as KAdam # type: ignore
from tensorflow.keras.losses import SparseCategoricalCrossentropy # type: ignore

# === PyTorch ===
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# === Sklearn ===
from sklearn.neural_network import MLPClassifier


def make_model(framework, seed):
    random.seed(seed)
    np.random.seed(seed)

    if framework == "keras":
        tf.random.set_seed(seed)
        model = Sequential([
            Dense(1024, activation='relu', kernel_initializer=GlorotUniform(seed), input_shape=(32*32*3,)),
            Dense(512,  activation='relu', kernel_initializer=GlorotUniform(seed)),
            Dense(256,  activation='relu', kernel_initializer=GlorotUniform(seed)),
            Dense(128,  activation='relu', kernel_initializer=GlorotUniform(seed)),
            Dense(10,   activation='softmax',  # ← salida softmax, 5.ª capa
                kernel_initializer=GlorotUniform(seed))
        ])
        return model


    elif framework == "torch":
        torch.manual_seed(seed)
        model = nn.Sequential(
            nn.Linear(3072, 1024), nn.ReLU(),   
            nn.Linear(1024, 512),  nn.ReLU(),   
            nn.Linear(512, 256),   nn.ReLU(),   
            nn.Linear(256, 128),   nn.ReLU(),   
            nn.Linear(128, 10),                
            nn.Softmax(dim=1)                  
        )
        return model


    elif framework == "sklearn":
        model = MLPClassifier(
            hidden_layer_sizes=(1024, 512, 256, 128),
            activation='relu',
            solver='adam',
            max_iter=1,
            random_state=seed
        )
        return model

    else:
        raise ValueError(f"Framework '{framework}' no soportado.")


In [13]:
import time
import numpy as np

def train_network(network, X, y, optimizer, learning_rate, batch_size, timeout):
    results = []
    start_time = time.time()

    print(f"🔍 Detectando tipo de red: {type(network)}")

    # ======================= KERAS =======================
    try:


        if hasattr(network, "train_on_batch") and hasattr(network, "compile"):
            print("✅ Red Keras detectada. Iniciando entrenamiento...")
            loss_fn = SparseCategoricalCrossentropy()
            opt = KAdam(learning_rate) if optimizer == 'adam' else KSGD(learning_rate)
            network.compile(optimizer=opt, loss=loss_fn)

            num_samples = X.shape[0]
            while True:
                if time.time() - start_time >= timeout - 0.1:
                    break
                idx = np.random.choice(num_samples, batch_size, replace=False)
                x_batch = X[idx]
                y_batch = y[idx]
                loss = network.train_on_batch(x_batch, y_batch)
                elapsed = round(time.time() - start_time, 4)
                results.append((elapsed, float(loss)))
            print("✅ Entrenamiento Keras terminado.")
            return results
    except Exception as e:
        print("❌ Error en bloque Keras:", e)

    # ======================= PYTORCH =======================
    try:


        if isinstance(network, nn.Module):
            print("✅ Red PyTorch detectada. Iniciando entrenamiento...")
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            network.to(device)
            network.train()

            X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
            y_tensor = torch.tensor(y, dtype=torch.long).to(device)

            opt = optim.Adam(network.parameters(), lr=learning_rate) if optimizer == 'adam' else optim.SGD(network.parameters(), lr=learning_rate)
            loss_fn = nn.CrossEntropyLoss()

            num_samples = X.shape[0]
            while True:
                if time.time() - start_time >= timeout - 0.1:
                    break
                idx = np.random.choice(num_samples, batch_size, replace=False)
                x_batch = X_tensor[idx]
                y_batch = y_tensor[idx]

                opt.zero_grad()
                output = network(x_batch)
                loss = loss_fn(output, y_batch)
                loss.backward()
                opt.step()

                elapsed = round(time.time() - start_time, 4)
                results.append((elapsed, float(loss.item())))
            print("✅ Entrenamiento PyTorch terminado.")
            return results
    except Exception as e:
        print("❌ Error en bloque PyTorch:", e)

    # ======================= SKLEARN =======================
    try:

        if isinstance(network, MLPClassifier):
            print("✅ Red sklearn detectada. Iniciando entrenamiento...")
            num_samples = X.shape[0]
            classes = np.unique(y)
            while True:
                if time.time() - start_time >= timeout - 0.1:
                    break
                idx = np.random.choice(num_samples, batch_size, replace=False)
                x_batch = X[idx]
                y_batch = y[idx]

                network.partial_fit(x_batch, y_batch, classes=classes)

                probs = network.predict_proba(x_batch)
                log_probs = np.log(probs + 1e-9)
                loss = -np.mean(log_probs[np.arange(len(y_batch)), y_batch])

                elapsed = round(time.time() - start_time, 4)
                results.append((elapsed, float(loss)))
            print("✅ Entrenamiento sklearn terminado.")
            return results
    except Exception as e:
        print("❌ Error en bloque sklearn:", e)

    print("⚠️ Ningún framework compatible fue detectado.")
    raise ValueError(f"Tipo de red no reconocido. Tipo recibido: {type(network)}")


In [14]:
import numpy as np
from tensorflow.keras.datasets import cifar10 # type: ignore

# =================== CARGAR CIFAR-10 ===================
(X_train, y_train), (_, _) = cifar10.load_data()

# Aplanar imágenes (de 32x32x3 a 3072)
X_train = X_train.reshape((X_train.shape[0], -1)).astype("float32") / 255.0
y_train = y_train.flatten()  # y es originalmente 2D, necesitamos 1D

# =================== USAR SOLO UN SUBCONJUNTO PARA PRUEBA ===================
X_subset = X_train[:5000]
y_subset = y_train[:5000]

# =================== CREAR MODELO ===================
model = make_model("keras", seed=42)

# =================== ENTRENAR ===================
results = train_network(
    network=model,
    X=X_subset,
    y=y_subset,
    optimizer="adam",
    learning_rate=0.001,
    batch_size=64,
    timeout=30  # Entrenar por 30 segundos
)

# =================== IMPRIMIR RESULTADOS ===================
for t, loss in results[:5]:  # Mostrar solo los primeros 5 pasos
    print(f"Time: {t}s | Loss: {loss:.4f}")

/home/pipe/anaconda3/envs/deeplearning/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


🔍 Detectando tipo de red: <class 'keras.src.models.sequential.Sequential'>
✅ Red Keras detectada. Iniciando entrenamiento...
✅ Entrenamiento Keras terminado.
Time: 5.3939s | Loss: 2.2922
Time: 5.4009s | Loss: 2.5453
Time: 5.4069s | Loss: 2.5869
Time: 5.4118s | Loss: 2.7256
Time: 5.4177s | Loss: 2.8739
